# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime
from sklearn.metrics import log_loss,make_scorer
from sklearn.model_selection import GridSearchCV

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Reading data

In [2]:
data = pd.read_csv('TrainingData.csv',index_col=[0])
testData = pd.read_csv('TestData.csv',index_col=[0])

D:\Study\Python38\lib\site-packages\IPython\core\interactiveshell.py:3050: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
data.head()

In [ ]:
testData.columns

In [ ]:
feature_labels

# Identify feature and target labels

In [3]:
target_labels = ['Function','Use','Object_Type','Position_Type','Operating_Status','Pre_K','Reporting','Sharing','Student_Type']
feature_labels = ['Object_Description', 'Text_2', 'SubFund_Description', 'Job_Title_Description', 'Text_3', 'Text_4',
                  'Sub_Object_Description', 'Location_Description', 'FTE', 'Function_Description', 'Facility_or_Department',
                  'Position_Extra', 'Total', 'Program_Description', 'Fund_Description', 'Text_1']

In [4]:
X = data[feature_labels]
y = data[target_labels]

In [5]:
num_feature_labels = list(X.select_dtypes(include='number').columns.values)
text_feature_labels = list(X.select_dtypes(exclude='number').columns.values)

In [6]:
num_feature_labels

['FTE', 'Total']

# Numerical predictor model generation

In [30]:
num_pipe = Pipeline([
    ('si',SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('mm',SimpleImputer())
])
num_model = Pipeline([
    ('cs', ColumnTransformer(remainder='drop',transformers=[
        ('nt',num_pipe,num_feature_labels)
    ])),
    ('predictor',DecisionTreeClassifier(max_depth=5))
])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [31]:
num_model.fit(X,y)

MemoryError: Unable to allocate 1.53 MiB for an array with shape (400277,) and data type int32

In [25]:
def predict_model(model, X_test, returnPred=False,write_prediction=False):
    X_test = testData[feature_labels]
    y_pred = np.concatenate(model.predict_proba(X_test),axis=1)
    y_pred_columns = [cat+"__"+scat 
                      for cat,scat in [(category,subcategory)
                                       for category, subCategory_arr in list(zip(y.columns, num_model['predictor'].classes_)) 
                                       for subcategory in subCategory_arr]]
    df_y_pred = pd.DataFrame(y_pred,index=X_test.index,columns=y_pred_columns)
    y_pred_columns_sorted = sorted(y_pred_columns)
    df_y_pred = df_y_pred[y_pred_columns_sorted]

    if(write_prediction == True):
        df_y_pred.to_csv(datetime.now().strftime("%d%m%Y%H%M%S")+'_Submission.csv',sep=',',index=True)
        
    if(returnPred == True):
        return df_y_pred

In [ ]:
y_predprob = predict_model(num_model, X, True)

In [ ]:
log_loss(y,y_pred)